In [2]:
import cudaq
import numpy as np
from itertools import product
from cudaq import spin

In [18]:
# Define Pauli matrices
paulis = {
    'I': np.array([[1, 0], [0, 1]], dtype=complex),
    'X': np.array([[0, 1], [1, 0]], dtype=complex),
    'Y': np.array([[0, -1j], [1j, 0]], dtype=complex),
    'Z': np.array([[1, 0], [0, -1]], dtype=complex)
}

p_spin_op = {
    'I': spin.i,
    'X': spin.x,
    'Y': spin.y,
    'Z': spin.z
}

p_spin = [spin.i, spin.x, spin.y, spin.z]

pauli_labels = list(paulis.keys())

def get_pauli_basis(n_qubits):
    """Generate tensor products of Pauli operators for n qubits."""
    basis = []
    labels = []
    for label in product(pauli_labels, repeat=n_qubits):
        op = paulis[label[0]]
        for l in label[1:]:
            op = np.kron(op, paulis[l])
        basis.append(op)
        labels.append(''.join(label))
    return labels, basis

def decompose_into_paulis(A):
    """Decompose Hermitian matrix A into Pauli basis."""
    n = int(np.log2(A.shape[0]))
    labels, basis = get_pauli_basis(n)
    coeffs = []
    for label, P in zip(labels, basis):
        # Coefficient: Tr(P†A) / 2^n (note P† = P for Pauli)
        coeff = np.trace(P.conj().T @ A) / (2 ** n)
        if not np.isclose(coeff, 0, atol=1e-10):
            coeffs.append((coeff, label))
    return coeffs

def to_spin_operator(coeffs, labels):
    operator = 0.0*spin.i(0)
    for i in range(1, len(labels[0])):
        operator *= spin.i(i)
    
    for i in range(len(labels)):
        ops = coeffs[i]
        for j in range(len(labels[i])):
            ops *= p_spin_op[labels[i][j]](j)
        operator += ops
    return operator
            
to_spin_operator([2.0, 1.0], ["II", "IZ"])      

In [22]:
@cudaq.kernel
def b(q: cudaq.qview, n_qubits: int):
    #q = cudaq.qvector(n_qubits)
    for i in range(n_qubits):
        cudaq.h(q[i]) 

@cudaq.kernel
def ansatz(q: cudaq.qview, n_qubits: int, n_layers: int, theta: list[float]):
    for i in range(n_layers):
        for j in range(n_qubits):
            ry(theta[i * n_qubits + j], q[j])
        for j in range(n_qubits - 1):
            if ((i + j) % 2) == 0:
                cx(q[j], q[j + 1])

swap_matrix = np.array([
    [1, 0, 0, 0],
    [0, 0, 1, 0],
    [0, 1, 0, 0],
    [0, 0, 0, 1]
])

# Register custom SWAP operation
cudaq.register_operation("custom_swap", swap_matrix)


@cudaq.kernel
def overlap_bpsi(A: list[int], n_qubits: int, n_layers: int, theta: list[float]):
    q = cudaq.qvector(2*n_qubits+1)
    ansatz(q, n_qubits, n_layers, theta)
    for i in range(n_qubits, 2*n_qubits):
        h(q[i])
    i = 0
    h(2*n_qubits)
    for pauli in A:
        p_spin[pauli].ctrl(q[2*n_qubits], q[i])
        i += 1
    
    
    




In [23]:
print(cudaq.draw(overlap_bpsi, [3, 3, 3, 3, 3, 3, 1, 3, 1, 3], 5, 2, [0.1]*10))

AttributeError: 'Subscript' object has no attribute 'id'

In [17]:
help(cudaq.pauli_word)

Help on class pauli_word in module cudaq.mlir._mlir_libs._quakeDialects.cudaq_runtime:

class pauli_word(pybind11_builtins.pybind11_object)
 |  The `pauli_word` is a thin wrapper on a Pauli tensor product string, e.g. `XXYZ` on 4 qubits.
 |  
 |  Method resolution order:
 |      pauli_word
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(...)
 |      __init__(*args, **kwargs)
 |      Overloaded function.
 |      
 |      1. .. function:: __init__(self: cudaq.mlir._mlir_libs._quakeDialects.cudaq_runtime.pauli_word) -> None
 |              :noindex:
 |      
 |      
 |      
 |      2. .. function:: __init__(self: cudaq.mlir._mlir_libs._quakeDialects.cudaq_runtime.pauli_word, arg0: str) -> None
 |              :noindex:
 |  
 |  ----------------------------------------------------------------------
 |  Static methods inherited from pybind11_builtins.pybind11_object:
 |  
 |  __new__(*args, **kwargs) from pybind11_builtins